Construct Entry | EC number | Sequence csv for erxprot as for CLEAN's datasets

In [42]:
from src.utils import load_json
from collections import defaultdict
import pandas as pd
import subprocess

In [45]:
# Create csv index in format of other datasets

f = load_json('../data/erxprot/sequences/sequences.json')
id2ec = defaultdict(list)
id2seq = {}
for ec, v in f.items():
    for elt in v:
        id = list(elt.keys())[0]
        seq = elt[id]
        id2ec[id].append(ec)
        id2seq[id] = seq

print(len(id2ec), len(id2seq))

data = []
for k in id2ec.keys():
    data.append([k, ';'.join(id2ec[k]), id2seq[k]])

header = ['Entry', 'EC number', 'Sequence']
df = pd.DataFrame(data, columns=header)
df.to_csv('../data/erxprot/erxprot.csv', index=False, sep='\t')

104200 104200


In [15]:
# # Re-name erxprot pt files ONLY RUN ONCE

# for dir in ['../data/erxprot/clean/', '../data/erxprot/esm/']:
#     for fn in os.listdir(dir):
#         id = fn.split('|')[0].strip()
#         cmd = ["mv", dir + fn, dir + id + '.pt']
#         subprocess.run(cmd)

In [50]:
# Overlap in enzymes

erx_csv = pd.read_csv('../data/erxprot/erxprot.csv', delimiter='\t')
erx_csv.set_index('Entry', inplace=True)

swissprot = pd.read_csv('../data/swissprot/swissprot.csv', delimiter='\t')
swissprot.set_index('Entry', inplace=True)

print(len(set(erx_csv.index) & set(swissprot.index)) / len(set(erx_csv.index) | set(swissprot.index)))
idx_intersect = list(set(erx_csv.index) & set(swissprot.index))

0.14423657628370382


In [55]:
# Get consistent / discrepant ecs, update erxport
# w/ swissprot ec + extras
cons = []
discrep = []

for idx in idx_intersect:
    swiss = swissprot.loc[idx, 'EC number']
    erx = erx_csv.loc[idx, 'EC number']
    if sorted(swiss.split(';')) == sorted(erx.split(';')):
        cons.append(idx)
    else:
        discrep.append(idx)
        erx_extra = list(set(erx.split(';')) - set(swiss.split(';')))
        updated = swiss.split(';') + erx_extra
        erx_csv.loc[idx, 'EC number'] = ';'.join(updated)


erx_csv.to_csv('../data/erxprot/erxprot.csv', sep='\t')

In [53]:
len(cons), len(discrep), len(cons) / len(idx_intersect), len(discrep) / len(idx_intersect)

(39912, 1883, 0.9549467639669817, 0.04505323603301831)